In [1]:
import json
from datetime import datetime
from itertools import combinations

from settings.env import env
from web3 import Web3

In [2]:
ethereum_web3 = Web3(Web3.HTTPProvider(env.mainnet_http_provider_url))
bsc_web3 = Web3(Web3.HTTPProvider(env.bsc_http_provider_url))
polygon_web3 = Web3(Web3.HTTPProvider(env.polygon_http_provider_url))

PROVIDERS = {
    'ethereum': ethereum_web3,
    'bsc': bsc_web3,
    'polygon': polygon_web3,
}

In [179]:
# tokens
with open('./defi/tokens.json') as f:
    chain_token_list = json.load(f)

# uniswap v2 Factory
with open('./defi/uniswap_factory.json') as f:
    uniswap_v2_factory_abi = json.load(f)

# uniswap v2 Pair
with open('./defi/uniswap_pair.json') as f:
    uniswap_v2_pair_abi = json.load(f)

# multichain pool addresses
with open('./defi/multichain_pools.json') as f:
    multichain_pools = json.load(f)
    
# symbiosis pool addresses
with open('./defi/symbiosis_pools.json') as f:
    symbiosis_pools = json.load(f)

In [4]:
uniswap_v2_factory_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
uniswap_v2_factory_contract = PROVIDERS['ethereum'].eth.contract(
    address=uniswap_v2_factory_address, abi=uniswap_v2_factory_abi
)

pancakeswap_factory_address = '0xcA143Ce32Fe78f1f7019d7d551a6402fC5350c73'
pancakeswap_factory_contract = PROVIDERS['bsc'].eth.contract(
    address=pancakeswap_factory_address, abi=uniswap_v2_factory_abi
)

quickswap_factory_address = '0x5757371414417b8C6CAad45bAeF941aBc7d3Ab32'
quickswap_factory_contract = PROVIDERS['polygon'].eth.contract(
    address=quickswap_factory_address, abi=uniswap_v2_factory_abi
)

CONTRACTS = {
    'uniswapv2': uniswap_v2_factory_contract,
    'pancakeswap': pancakeswap_factory_contract,
    'quickswap': quickswap_factory_contract,
}

PAIRS = {
    'uniswapv2': uniswap_v2_pair_abi,
    'pancakeswap': uniswap_v2_pair_abi,
    'quickswap': uniswap_v2_pair_abi,
}

minABI = [
    {
        'constant': True,
        'inputs': [{'name': "_owner", 'type': "address"}],
        'name': "balanceOf",
        'outputs': [{'name': "balance", 'type': "uint256"}],
        'type': "function",
    },
]

In [181]:
symbiosis_ABI = [
    {
        'constant': True,
        'inputs': [{'name': "_owner", 'type': "address"}],
        'name': "getTokenIndex",
        'outputs': [{'name': "index", 'type': "uint8"}],
        'type': "function",
    },
    {
        'constant': True,
        'inputs': [{'name': "_owner", 'type': "uint8"}],
        'name': "getTokenBalance",
        'outputs': [{'name': "index", 'type': "uint256"}],
        'type': "function",
    }
]

In [195]:
response = {"swap_pools": {}, "bridge_pools": {}}

In [184]:
for chainPair in symbiosis_pools:

    protocol_name = chainPair['protocolName']
    chain_name = chainPair['chainPair']
    chain_result = []
    
    for token_info in chainPair['nerves']:
        token_address = Web3.toChecksumAddress(token_info['address'])
        token_result = {}

        try:
            
#             if chain_name.split('/')[0] == 'bsc':
#                 ABI_get = requests.get("https://api.bscscan.com/api?module=contract&action=getabi&address={}".format(token_address))
#                 r = ABI_get.json()
#                 ABI = json.loads(r['result'])
                
            contract = PROVIDERS[chain_name.split('_')[0]].eth.contract(address=token_address, abi=symbiosis_ABI)
            
            token0address = Web3.toChecksumAddress(token_info['tokens'][0]['address'])
            token0decimals = token_info['tokens'][0]['decimals']
            token1address = Web3.toChecksumAddress(token_info['tokens'][1]['address'])
            token1decimals = token_info['tokens'][1]['decimals']
            
            ind0 = contract.functions.getTokenIndex(token0address).call()
            ind1 = contract.functions.getTokenIndex(token1address).call()
            
            token_balance0 = contract.functions.getTokenBalance(ind0).call() / (10 ** token0decimals)
            token_balance1 = contract.functions.getTokenBalance(ind1).call() / (10 ** token1decimals)
            
            token_balance = token_balance0 + token_balance1            
            now = datetime.now()

            token_result['protocol_name'] = protocol_name
            token_result['pair_address'] = token_address
            token_result['token_supply'] = token_balance
            token_result['date_updated'] = now.isoformat()
            chain_result.append(token_result)
                
        except Exception as e:
            print(f'Failed to update {protocol_name} pool info on {chain_name}: {e}')
                
    response["bridge_pools"][chain_name] = chain_result

In [185]:
response

{'swap_pools': {},
 'bridge_pools': {'bsc/ethereum': [{'protocol_name': 'symbiosis',
    'pair_address': '0xab0738320A21741f12797Ee921461C691673E276',
    'token_supply': 1032074.3007520527,
    'date_updated': '2022-11-20T23:31:18.268110'}],
  'polygon/ethereum': [{'protocol_name': 'symbiosis',
    'pair_address': '0xab0738320A21741f12797Ee921461C691673E276',
    'token_supply': 512465.74234500004,
    'date_updated': '2022-11-20T23:31:19.499821'}],
  'polygon/bsc': [{'protocol_name': 'symbiosis',
    'pair_address': '0xF4BFF06E02cdF55918e0ec98082bDE1DA85d33Db',
    'token_supply': 121716.40375339272,
    'date_updated': '2022-11-20T23:31:20.677334'}]}}

In [196]:
def get_pools():
    response = {"swap_pools": {}, "bridge_pools": {}}

    for chain in chain_token_list:
        print('check1')
        chain_name = chain['chain']
        protocol_name = chain['protocolName']

        chain_result = []
        response["swap_pools"][chain_name] = chain_result

        web3 = PROVIDERS.get(chain_name, None)
        contract = CONTRACTS.get(protocol_name, None)
        pair_abi = PAIRS.get(protocol_name, None)

        if not chain_name:
            print(f'No web3 provider found for {chain_name}')
            continue

        if not contract:
            print(f'No factory contract found for chain {chain_name}: {protocol_name}')
            continue

        if not pair_abi:
            print(f'No pair ABI found for chain {chain_name}: {protocol_name}')
            continue

        pairs = combinations(chain['tokens'], 2)
        for pair in pairs:
            try:
                t1, t2 = pair
                t1_name = t1['name']
                t2_name = t2['name']
                name = f'{t1_name}/{t2_name}'

                pair_address = contract.functions.getPair(t1['address'], t2['address']).call()
                pair_contract = web3.eth.contract(address=pair_address, abi=pair_abi)

                t1_address = pair_contract.functions.token0().call()
                t2_address = pair_contract.functions.token1().call()

                now = datetime.now()

                t1_supply, t2_supply, _ = pair_contract.functions.getReserves().call()

                if t1_address != t1['address']:
                    name = f'{t2_name}/{t1_name}'

                chain_result.append({
                    'protocol_name': protocol_name,
                    'pair_name': name,
                    'token_0': t1_address,
                    'token_1': t2_address,
                    'pair_address': pair_address,
                    'token_0_supply': t1_supply,
                    'token_1_supply': t2_supply,
                    'date_updated': now.isoformat()
                })

            except Exception as e:
                # do not add pair to chain list if cannot get info
                print(f'Failed to update pair {name} on {chain_name}: {e}')

    # multichain pools
    for chain in multichain_pools:
        print('check2')
        protocol_name = chain['protocolName']
        chain_name = chain['chain']
        chain_result = []
        temp_chain_token_info = {}
        for chain_tokens in chain_token_list:
            if chain_tokens['chain'] == chain_name:
                temp_chain_token_info = chain_tokens['tokens']
        for bridge_token_info in chain['tokens']:
            token_result = {}
            for i in temp_chain_token_info:
                if i['name'] == bridge_token_info['name']:
                    token_address = i['address']
                name = bridge_token_info['name']
            try:
                contract = PROVIDERS[chain_name].eth.contract(address=token_address, abi=minABI)
                token_balance = contract.functions.balanceOf(bridge_token_info['address']).call()
                now = datetime.now()

                token_result['protocol_name'] = protocol_name
                token_result['token_name'] = name
                token_result['token_address'] = token_address
                token_result['token_supply'] = token_balance
                token_result['date_updated'] = now.isoformat()
                chain_result.append(token_result)
            except Exception as e:
                # do not add pair to chain list if cannot get info
                print(f'Failed to update {protocol_name} pool info of {name} on {chain_name}: {e}')
        response["bridge_pools"][chain_name] = chain_result
    
    # symbiosis pools
    
    for chainPair in symbiosis_pools:
        
        print('check3')

        protocol_name = chainPair['protocolName']
        chain_name = chainPair['chainPair']
        chain_result_sym = []
    
        for token_info in chainPair['nerves']:
            token_address = Web3.toChecksumAddress(token_info['address'])
            token_result = {}

            try:
                
                contract = PROVIDERS[chain_name.split('_')[0]].eth.contract(address=token_address, abi=symbiosis_ABI)
            
                token0address = Web3.toChecksumAddress(token_info['tokens'][0]['address'])
                token0decimals = token_info['tokens'][0]['decimals']
                token1address = Web3.toChecksumAddress(token_info['tokens'][1]['address'])
                token1decimals = token_info['tokens'][1]['decimals']
            
                ind0 = contract.functions.getTokenIndex(token0address).call()
                ind1 = contract.functions.getTokenIndex(token1address).call()
            
                token_balance0 = contract.functions.getTokenBalance(ind0).call() / (10 ** token0decimals)
                token_balance1 = contract.functions.getTokenBalance(ind1).call() / (10 ** token1decimals)
            
                token_balance = token_balance0 + token_balance1            
                now = datetime.now()

                token_result['protocol_name'] = protocol_name
                token_result['pair_address'] = token_address
                token_result['token_supply'] = token_balance
                token_result['date_updated'] = now.isoformat()
                chain_result_sym.append(token_result)
                
            except Exception as e:
                print(f'Failed to update {protocol_name} pool info on {chain_name}: {e}')
        print('check')       
        response["bridge_pools"][chain_name] = chain_result_sym

    return response



In [197]:
response = get_pools()

check1
check1
check1
check2
check2
check2
check3
check
check3
check
check3
check


In [198]:
response

{'swap_pools': {'ethereum': [{'protocol_name': 'uniswapv2',
    'pair_name': 'WBTC/WETH',
    'token_0': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
    'token_1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
    'pair_address': '0xBb2b8038a1640196FbE3e38816F3e67Cba72D940',
    'token_0_supply': 22324172815,
    'token_1_supply': 3176262736593291545279,
    'date_updated': '2022-11-21T16:15:44.034171'},
   {'protocol_name': 'uniswapv2',
    'pair_name': 'DAI/WETH',
    'token_0': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
    'token_1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
    'pair_address': '0xA478c2975Ab1Ea89e8196811F51A7B7Ade33eB11',
    'token_0_supply': 6262676491699806205054213,
    'token_1_supply': 5543519200528614370912,
    'date_updated': '2022-11-21T16:15:47.387267'},
   {'protocol_name': 'uniswapv2',
    'pair_name': 'USDC/WETH',
    'token_0': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    'token_1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
    'pa

In [18]:
import asyncio
import json
from datetime import datetime
from enum import Enum

import aioredis
import httpx

from defi.pools import get_pools
from settings.env import env
from pycoingecko import CoinGeckoAPI

In [19]:

redis = aioredis.from_url(
    env.redis_dsn,
    encoding="utf-8",
    decode_responses=True,
)



In [20]:
class OwlracleGasPaths(str, Enum):
    bsc: str = f'https://api.owlracle.info/v3/bsc/gas?accept=90&apikey={env.owlracle_api_key}'
    ethereum: str = f'https://api.owlracle.info/v3/eth/gas?accept=90&apikey={env.owlracle_api_key}'
    polygon: str = f'https://api.owlracle.info/v3/poly/gas?accept=90&apikey={env.owlracle_api_key}'

    @classmethod
    def names(cls):
        return [item.name for item in cls]

In [27]:
async def get_estimated_fee(url):
    task = asyncio.create_task(collect_quotes_for_gas_coingecko())
    quotes = await asyncio.gather(task)
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url)
            for item in quotes[0]:
                if url.split('/')[-2] == 'bsc' and item['chain'] == 'bsc':
                    return {
                    'gwei': response.json()['speeds'][0]['gasPrice'],
                    'tokenPrice': item['value'],
                    'dateUpdated': datetime.now().isoformat(),
                    }
                elif url.split('/')[-2] == item['chain']:
                    return {
                    'gwei': response.json()['speeds'][0]['baseFee'],
                    'tokenPrice': item['value'],
                    'dateUpdated': datetime.now().isoformat(),
                    }
        except (KeyError, httpx.NetworkError, httpx.HTTPError) as e:
            print(f'Failed to get gas ({url}): {e}')
            return None

In [28]:
tasks = [asyncio.create_task(get_estimated_fee(path.value)) for path in OwlracleGasPaths]
estimated_fee_results = await asyncio.gather(*tasks)

In [29]:
estimated_fee_results

[{'gwei': 5,
  'tokenPrice': 273.52,
  'dateUpdated': '2022-11-20T11:25:54.941301'},
 {'gwei': 12.284427703,
  'tokenPrice': 1219.67,
  'dateUpdated': '2022-11-20T11:25:55.303402'},
 {'gwei': 1.01e-07,
  'tokenPrice': 0.872848,
  'dateUpdated': '2022-11-20T11:25:55.303937'}]

In [225]:
async def get_gas() -> None:
    tasks = [asyncio.create_task(get_estimated_fee(path.value)) for path in OwlracleGasPaths]
    estimated_fee_results = await asyncio.gather(*tasks)
    new_gas = dict(zip(OwlracleGasPaths.names(), estimated_fee_results))
    print(new_gas)
#     old_gas = await redis.get('gas')

#     if old_gas:
#         try:
#             old_gas = json.loads(old_gas)
#             old_values = [x for x in old_gas if new_gas.get(x) is None or not new_gas.get(x, {}).get('value')]
#             for key in old_values:
#                 new_gas[key] = old_gas[key]

#         except Exception as e:
#             print(f'Failed to save gas: {e}')

#     await redis.set('gas', json.dumps(new_gas))

In [226]:
task = asyncio.create_task(get_gas())
gas = await asyncio.gather(task)

{'bsc': {'gwei': 5, 'tokenPrice': 275.7, 'dateUpdated': '2022-11-14T19:56:29.089174'}, 'ethereum': {'gwei': 23.107961757, 'tokenPrice': 1236.57, 'dateUpdated': '2022-11-14T19:56:29.103878'}, 'polygon': {'gwei': 14.519417979, 'tokenPrice': 0.912869, 'dateUpdated': '2022-11-14T19:56:29.106786'}}


In [23]:
async def collect_quotes_coingecko():
    cg = CoinGeckoAPI()
    ids_q = ['usd-coin', 'dai', 'ethereum', 'tether', 'wrapped-bitcoin']
    temp_quotes = cg.get_price(ids=ids_q, vs_currencies='usd')
    quotes_output = [
        {"name": str('DAI'), "value": temp_quotes['dai']['usd']},
        {"name": str('USDC'), "value": temp_quotes['usd-coin']['usd']},
        {"name": str('USDT'), "value": temp_quotes['tether']['usd']},
        {"name": str('WETH'), "value": temp_quotes['ethereum']['usd']},
        {"name": str('WBTC'), "value": temp_quotes['wrapped-bitcoin']['usd']}
    ]
    return quotes_output

In [24]:
async def collect_quotes_for_gas_coingecko():
    cg = CoinGeckoAPI()
    ids_q = ['ethereum', 'matic-network', 'binancecoin']
    temp_quotes = cg.get_price(ids=ids_q, vs_currencies='usd')
    quotes_for_gas_output = [
        {"name": str('ETH'), "value": temp_quotes['ethereum']['usd'], "chain": str('eth')},
        {"name": str('MATIC'), "value": temp_quotes['matic-network']['usd'], "chain": str('poly')},
        {"name": str('BNB'), "value": temp_quotes['binancecoin']['usd'], "chain": str('bsc')}
    ]
    return quotes_for_gas_output

In [25]:
task = asyncio.create_task(collect_quotes_for_gas_coingecko())
quotes = await asyncio.gather(task)

In [26]:
quotes[0]

[{'name': 'ETH', 'value': 1219.67, 'chain': 'eth'},
 {'name': 'MATIC', 'value': 0.872848, 'chain': 'poly'},
 {'name': 'BNB', 'value': 273.52, 'chain': 'bsc'}]

In [120]:
# cg = CoinGeckoAPI()
# cg.get_coins_markets(vs_currency='usd')

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579',
  'current_price': 16794.19,
  'market_cap': 322271023441,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 352355010328,
  'total_volume': 29950776486,
  'high_24h': 16911.74,
  'low_24h': 15906.19,
  'price_change_24h': 264.44,
  'price_change_percentage_24h': 1.59981,
  'market_cap_change_24h': 4689007094,
  'market_cap_change_percentage_24h': 1.47647,
  'circulating_supply': 19207025.0,
  'total_supply': 21000000.0,
  'max_supply': 21000000.0,
  'ath': 69045,
  'ath_change_percentage': -75.72653,
  'ath_date': '2021-11-10T14:24:11.849Z',
  'atl': 67.81,
  'atl_change_percentage': 24615.83482,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2022-11-14T09:55:38.630Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595

In [30]:
async def get_quotes() -> None:
    task = asyncio.create_task(collect_quotes_coingecko())
    new_quotes = await asyncio.gather(task)
    old_quotes = await redis.get('quotes')

    await redis.set('quotes', json.dumps(new_quotes[0]))


async def get_gas_scheduler() -> None:
    while True:
        await asyncio.gather(
            get_gas(),
            asyncio.sleep(env.get_gas_delay),
        )


async def get_quotes_scheduler() -> None:
    while True:
        await asyncio.gather(
            get_quotes(),
            asyncio.sleep(env.get_quotes_delay),
        )


async def get_and_store_pools() -> None:
    new_pools = get_pools()

    old_pools = await redis.get('pools')

    if not old_pools:
        pools = new_pools

    else:
        try:
            old_pools = json.loads(old_pools)

        except Exception:
            pass
        for pool_type in ["swap_pools", "bridge_pools"]:
            for chain_name, old_chain_pairs in old_pools[pool_type].items():
                if pool_type == "swap_pools":
                    key_name = 'pair_name'
                else:
                    key_name = 'token_name'
                old_chain_pairs_dict = dict((item[key_name], item) for item in old_chain_pairs)
                new_chain_pairs = dict((item[key_name], item) for item in new_pools[pool_type][chain_name])

                not_updated = [item for name, item in old_chain_pairs_dict.items() if name not in new_chain_pairs]
                new_pools[pool_type][chain_name] += not_updated

        pools = new_pools

    await redis.set('pools', json.dumps(pools))


async def get_pools_scheduler() -> None:
    while True:
        await asyncio.gather(
            get_and_store_pools(),
            asyncio.sleep(env.get_pools_delay),
        )


async def main():
    while True:
        await asyncio.gather(get_gas_scheduler(), get_pools_scheduler(), get_quotes_scheduler())


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [199]:
import requests
import json

In [202]:
url = 'http://localhost:8000/api/v1/'

x = requests.get(url)

print(x)

<Response [200]>


In [203]:
x.text

'{"pools":{"swap_pools":{"ethereum":[{"protocolName":"uniswapv2","pairName":"WBTC/WETH","token0":"0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599","token1":"0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2","pairAddress":"0xBb2b8038a1640196FbE3e38816F3e67Cba72D940","token0Supply":"22193874158","token1Supply":"3195217535284527071157","dateUpdated":"2022-11-21T20:01:04.637238"},{"protocolName":"uniswapv2","pairName":"DAI/WETH","token0":"0x6B175474E89094C44Da98b954EedeAC495271d0F","token1":"0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2","pairAddress":"0xA478c2975Ab1Ea89e8196811F51A7B7Ade33eB11","token0Supply":"6143383620482093694567827","token1Supply":"5652632725982248415237","dateUpdated":"2022-11-21T20:01:05.898555"},{"protocolName":"uniswapv2","pairName":"USDC/WETH","token0":"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48","token1":"0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2","pairAddress":"0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc","token0Supply":"40655871736902","token1Supply":"3726902252607543482426

In [204]:
res = json.loads(x.text)

In [205]:
res

{'pools': {'swap_pools': {'ethereum': [{'protocolName': 'uniswapv2',
     'pairName': 'WBTC/WETH',
     'token0': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
     'token1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
     'pairAddress': '0xBb2b8038a1640196FbE3e38816F3e67Cba72D940',
     'token0Supply': '22193874158',
     'token1Supply': '3195217535284527071157',
     'dateUpdated': '2022-11-21T20:01:04.637238'},
    {'protocolName': 'uniswapv2',
     'pairName': 'DAI/WETH',
     'token0': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
     'token1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
     'pairAddress': '0xA478c2975Ab1Ea89e8196811F51A7B7Ade33eB11',
     'token0Supply': '6143383620482093694567827',
     'token1Supply': '5652632725982248415237',
     'dateUpdated': '2022-11-21T20:01:05.898555'},
    {'protocolName': 'uniswapv2',
     'pairName': 'USDC/WETH',
     'token0': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
     'token1': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756